In [ ]:
import os
import pandas as pd

In [ ]:
#Read in the data from cvs file
df = pd.read_csv("btc.csv", index_col=0)

In [ ]:
#Calculate the 50 day and 100 day moving averages
df['50 Day MA'] = df['Close'].rolling(window=50).mean()
df['100 Day MA'] = df['Close'].rolling(window=100).mean()

In [ ]:
#Set index to the Date column
df.index = pd.to_datetime(df.index)

In [ ]:
#Remove Excess Columns with missing values
del df["Dividends"]
del df["Stock Splits"]

In [ ]:
df.columns = [c.lower() for c in df.columns]

In [ ]:
#Set Tomorrow
df["tomorrow"] = df["close"].shift(-1)

In [ ]:
#Set Target data
df["target"] = (df["tomorrow"] > df["close"]).astype(int)

In [ ]:
df["target"].value_counts()

1    1674
0    1473
Name: target, dtype: int64

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#Utlize Random Forest
model = RandomForestClassifier(n_estimators=150, min_samples_split=50, random_state=1)

train = df.iloc[:-200].dropna()
test = df.iloc[-200:]

values = ["close", "open", "high", "low", "50 day ma", "100 day ma"]
model.fit(train[values], train["target"])

RandomForestClassifier(min_samples_split=50, n_estimators=150, random_state=1)

In [ ]:
from sklearn.metrics import precision_score

#Output score
predictionsRFC = model.predict(test[values])
predictionsRFC = pd.Series(predictionsRFC, index=test.index)
precision_score(test["target"], predictionsRFC)

0.5298507462686567

In [ ]:
#Output the predictions and the target data
predictionsRFC, test["target"]

(Date
 2022-10-12 00:00:00+00:00    1
 2022-10-13 00:00:00+00:00    1
 2022-10-14 00:00:00+00:00    0
 2022-10-15 00:00:00+00:00    1
 2022-10-16 00:00:00+00:00    1
                             ..
 2023-04-25 00:00:00+00:00    0
 2023-04-26 00:00:00+00:00    0
 2023-04-27 00:00:00+00:00    1
 2023-04-28 00:00:00+00:00    1
 2023-04-30 00:00:00+00:00    1
 Length: 200, dtype: int64,
 Date
 2022-10-12 00:00:00+00:00    1
 2022-10-13 00:00:00+00:00    0
 2022-10-14 00:00:00+00:00    0
 2022-10-15 00:00:00+00:00    1
 2022-10-16 00:00:00+00:00    1
                             ..
 2023-04-25 00:00:00+00:00    1
 2023-04-26 00:00:00+00:00    1
 2023-04-27 00:00:00+00:00    0
 2023-04-28 00:00:00+00:00    0
 2023-04-30 00:00:00+00:00    0
 Name: target, Length: 200, dtype: int64)

In [ ]:
from xgboost import XGBClassifier

#Utlize XGBoost 
modelXGB = XGBClassifier(learning_rate=.1, n_estimators=100)

modelXGB.fit(train[values], train["target"])

predictions = modelXGB.predict(test[values])
predictions = pd.Series(predictions, index=test.index)
precision_score(test["target"], predictions)

0.5288461538461539